In [1]:
# Importing modules

import socks
import socket
import requests
import codecs
from bs4 import BeautifulSoup

## Data Extraction from HTML

In [46]:
# Extract useful data from each post
dates = []
name = []

file = codecs.open(f"../raw-html/hive.html", "r", "utf-8")
index = file.read()
soup = BeautifulSoup(index, 'lxml')
    
post_cards = soup.find_all("div", {"class": "blog-card-main"})
for card in post_cards:
    title = card.findChildren("h2", recursive=True)
    name.append(title[0].text)
    
post_dates_tag = soup.find_all("div", {"class": "blog-card-time-start"})
post_dates_p = [div.findChildren("p" , recursive=False) for div in post_dates_tag]
desc_p = [j for sub, i in post_dates_p for j in sub]
post_dates = [tag.strip() for tag in desc_p[1::2]]
dates.extend(post_dates)

In [47]:
print(dates)
print(name)
print(desc)

['January 2022', 'September 2021', 'June 2022', 'July 2021', 'May 2022', 'May 2022', 'January 2022', 'July 2021', 'November 2021', 'November 2021', 'November 2021', 'July 2022', 'June 2022', 'July 2021', 'July 2021', 'June 2022', 'December 2021', 'April 2022', 'May 2022', 'November 2021', 'September 2021', 'December 2021', 'June 2022', 'May 2022', 'June 2022', 'June 2021', 'November 2021', 'December 2021', 'May 2022', 'April 2022', 'April 2022', 'July 2021', 'March 2022', 'November 2021', 'November 2021', 'July 2021', 'July 2022', 'October 2021', 'August 2021', 'April 2022', 'February 2022', 'June 2022', 'October 2021', 'August 2022', 'July 2022', 'May 2022', 'July 2021', 'September 2021', 'June 2022', 'June 2022', 'October 2021', 'August 2021', 'May 2022', 'May 2022', 'July 2021', 'February 2022', 'November 2021', 'July 2021', 'April 2022', 'March 2022', 'April 2022', 'October 2021', 'July 2022', 'July 2022', 'February 2022', 'July 2021', 'January 2022', 'September 2022', 'October 202

In [48]:
print(len(dates) == len(name))

True


In [57]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['Company Name', 'Location', 'Industry', 'Post Date'])
df['Company Name'] = name
df['Post Date'] = dates

df

,Company Name,Location,Industry,Post Date
0,Otto Dörner GmbH & Co. KG,NaN,NaN,January 2022
1,APR Supply,NaN,NaN,September 2021
2,MHIRE,NaN,NaN,June 2022
3,RIVADIS,NaN,NaN,July 2021
4,KDE,NaN,NaN,May 2022
...,...,...,...,...
154,Instituto De Gesto Estratégica De Sade Do Dist...,NaN,NaN,November 2021
155,Arte Radiotelevisivo Argentino (Artear),NaN,NaN,June 2022
156,Steven L. Sugarman & Associates,NaN,NaN,July 2021
157,Rodonaves Transportes E Encomendas Ltda,NaN,NaN,January 2022


### Drop posts not in 2022

In [58]:
index = []
for i in range(len(df)):
    if '2021' in df['Post Date'][i]:
        index.append(i)

df = df.drop(index)
df = df.reset_index(drop=True)
df

,Company Name,Location,Industry,Post Date
0,Otto Dörner GmbH & Co. KG,NaN,NaN,January 2022
1,MHIRE,NaN,NaN,June 2022
2,KDE,NaN,NaN,May 2022
3,NETWORK4CARS,NaN,NaN,May 2022
4,Banco Caribe,NaN,NaN,January 2022
...,...,...,...,...
80,AdaptIT,NaN,NaN,July 2022
81,Exela Technologies,NaN,NaN,June 2022
82,Arte Radiotelevisivo Argentino (Artear),NaN,NaN,June 2022
83,Rodonaves Transportes E Encomendas Ltda,NaN,NaN,January 2022


## Selenium automation

In [59]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Searching in Glassdoor

In [64]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') 
options.add_argument("enable-automation");
# options.add_argument("--headless");
options.add_argument("--no-sandbox");
options.add_argument("--disable-extensions");
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");

path = 'D:/Downloads/chromedriver'
driver = webdriver.Chrome(options=options, executable_path=path)

for i in range(73, len(df)):
    try:
        company = df['Company Name'][i]
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' glassdoor "overview"')
        search.send_keys(Keys.RETURN)
        
        # Delay to be able to solve Google captcha
        if i == 73: time.sleep(20)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" not in driver.current_url:
            print('none')
            continue
        
        pagesource = driver.page_source
        soup = BeautifulSoup(pagesource, 'html.parser')
        company_location = company_industry = '-'
        
        labels = soup.find_all('label')
        for label in labels:
            if label.text == "Headquarters:":
                company_location = label.next_sibling.text
                df['Location'][i] = company_location
            elif (label.text == "Industry:"):
                company_industry = label.next_sibling.text
                df['Industry'][i] = company_industry

        print(f'{i}: {company}, {company_location}, {company_industry}')
        
    except:
        print("none")
    
driver.quit()

73: RateGain, Noida, India, Computer Hardware Development
74: Friedrich, , Civic, Welfare & Social Services
75: GomeA, Kuala Lumpur, Malaysia, -
76: EBM, Münchenstein, Switzerland, -
77: Hyundai Samho Heavy Industries Co.,Ltd. (South Korea), Ulsan, South Korea, Transportation Equipment Manufacturing
78: Diskriter, Pittsburgh, PA, Healthcare Services & Hospitals
79: Emil Frey, Zürich, Switzerland, Motor Vehicle Dealers
80: AdaptIT, Durban, South Africa, Information Technology Support Services
81: Exela Technologies, Irving, TX, Information Technology Support Services
82: Arte Radiotelevisivo Argentino (Artear), Buenos Aires, Argentina, -
83: Rodonaves Transportes E Encomendas Ltda, -, -
84: Centerline Communication Llc, West Bridgewater, MA, Telecommunications Services


In [65]:
df.to_csv('hive_data.csv')